In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
page = 1

# Цена

In [4]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

# Таблица

In [5]:
def getTable(flat_page):
    table = flat_page.find('table',attrs={'class':'object_descr_props'})
    table=html_stripper(table)
    return table

# Этаж, кол-во этажей

In [6]:
def getFloor(table):
    floor=re.split('Этаж:|Тип дома:', table)[1]
    floor=re.split(' |\n\n|\n|\n\n\n ',str(floor))
    k=''
    for i in range(0,len(floor)-1):
        if (floor[i]!=''): 
            k=k+floor[i]
    k=re.split('xa0|/' , str(k),maxsplit=1)
    if (len(k)==1):
        return int(k[0]), int(k[0])
    else:
        return int(k[0]), int(k[1])

# Общая площадь

In [7]:
def getTotsp(table):
    Totsp=re.split('Общая площадь:|Площадь комнат:', table)[1]
    Totsp=re.split('\n|\xa0|м2', str(Totsp))
    k=0
    kol=0
    for i in range(0,len(Totsp[2])):
        if (Totsp[2][i].isdigit()): 
            k=k*10+int(Totsp[2][i])
        else:
            kol=kol+1
    return(k/pow(10,kol))

# Жилая площадь

In [8]:
def getLivesp(table):
    Livesp=re.split('Жилая площадь:|Площадь кухни:',table)[1]
    Livesp=re.split('\n|\xa0|м2|,', str(Livesp))
    k=0
    kol=-1
    for i in range(0,len(Livesp)-1):
        if (Livesp[i].isdigit()):
            k=k*10+int(Livesp[i])
            kol=kol+1
    return (k/pow(10,kol))

# Площадь кухни 

In [9]:
def getKitsp(table):
    Kitsp=re.split('Площадь кухни:| санузлов:', table)[1]
    Kitsp=re.split('\n|\xa0|м2|,', str(Kitsp))
    k=0
    kol=-1
    for i in range(0,len(Kitsp)):
        if (Kitsp[i].isdigit()):
            k=k*10+int(Kitsp[i])
            kol=kol+1
    return (k/pow(10,kol))

# Новостройка, монолитный

In [10]:
def getBrick(table):
    brick=re.split('Тип дома:|Тип продажи:', table)[1]
    novostr=''
    novostr=re.search(r'новостройка', str(brick))
    if (novostr!=None):
        New=1
    else: 
        New=0
    monolit=re.search(r'монолитный', str(brick))
    kirpich=re.search(r'кирпичный', str(brick))
    if ((monolit!=None)or(kirpich!=None)):
        k=1
    else:
        k=0
    return New, k

# Балкон

In [29]:
def getBalkon(table):
    Bal=re.split('Балкон:|Лифт:',table)[1]
    balkon=re.search(r'балк', str(Bal))
    lodg=re.search(r'лодж',str(Bal))
    if (balkon!=None)or(lodg!=None):
        k=1
    else:
        k=0
    return k

# Количетво комнат

In [12]:
def getRooms(flat_page):
    rooms=flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms=html_stripper(rooms)
    rooms=re.split('\n|xa0| |\n\n|-комн.', str(rooms))
    k=0
    for i in range(0,len(rooms)-1):
        if 'кв' in rooms[i]:
            break
        if rooms[i].isdigit():
            k=int(rooms[i])
    return k

# Минут до метро, пешком/транспортом

In [13]:
def getMetrdist(flat_page):
    metro=flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metro=html_stripper(metro)
    walk=re.search(r'пешком', str(metro))
    if (walk!=None):
        k=1
    else:
        k=0
    metro=re.split('мин.', str(metro))[0]
    n=0
    for i in range(0,len(metro)-1):
        if metro[i].isdigit():
            n=n*10+int(metro[i])
    return n,k

# Расстояние до центра

In [26]:
def getDist(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    lat=math.radians(lat)
    lon=math.radians(lon)
    lat0=math.radians(55.7537090377)
    lon0=math.radians(37.61981338)
    S=6378.1*math.acos(math.sin(lat)*math.sin(lat0)+math.cos(lat)*math.cos(lat0)*math.cos(lon0-lon))
    return S

# Парсинг

In [31]:
data=pd.DataFrame(columns=['District','Link','Price','Rooms','Totsp','Livesp','Kitsp','floor','number of floors','MetrDist','Walk','New','Brick','Balkon','DistCenter'])

In [32]:
district_url = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district[0]=39&district[10]=49&district[11]=50&district[12]=51&district[13]=52&district[14]=53&district[15]=54&district[16]=55&district[1]=40&district[2]=41&district[3]=42&district[4]=43&district[5]=44&district[6]=45&district[7]=46&district[8]=47&district[9]=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1']

In [33]:
district=['ЦАО','САО','СВАО','ВАО','ЮВАО','ЮАО','ЮЗАО','ЗАО','СЗАО']

In [34]:
data

,District,Link,Price,Rooms,Totsp,Livesp,Kitsp,floor,number of floors,MetrDist,Walk,New,Brick,Balkon,DistCenter


In [35]:
flatStats= {'District':0}

In [36]:
for j in range(0,9):
    page=1
    links = []
    for page in range(1, 30):
        page_url =  district_url[j].format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    for i in range(0,len(links)):
        flat_url=''
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
        table=getTable(flat_page)
        flatStats['Link']=links[i]
        flatStats['Price']=getPrice(flat_page)
        flatStats['Rooms']=getRooms(flat_page)
        flatStats['Totsp']=getTotsp(table)
        flatStats['Livesp']=getLivesp(table)
        flatStats['Kitsp']=getKitsp(table)
        p=getFloor(table)
        flatStats['floor']=p[0]
        flatStats['number of floors']=p[1]
        g=getMetrdist(flat_page)
        flatStats['MetrDist']=g[0]
        flatStats['Walk']=g[1]
        e=getBrick(table)
        flatStats['New']=e[0]
        flatStats['Brick']=e[1]
        flatStats['Balkon']=getBalkon(table)
        flatStats['DistCenter']=getDist(flat_page)
        flatStats['District']=district[j]
        data=data.append(flatStats, ignore_index=True)

In [37]:
data

,District,Link,Price,Rooms,Totsp,Livesp,Kitsp,floor,number of floors,MetrDist,Walk,New,Brick,Balkon,DistCenter
0,ЦАО,150882390,40910468.0,2.0,45.6,25.0,12.0,6.0,7.0,2.0,1.0,1.0,1.0,0.0,0.855920
1,ЦАО,48903041,12149428.0,1.0,38.6,0.0,0.0,2.0,7.0,5.0,1.0,1.0,1.0,0.0,2.923647
2,ЦАО,149623161,10950000.0,2.0,65.2,36.2,7.2,3.0,10.0,5.0,1.0,0.0,0.0,0.0,6.024716
3,ЦАО,149031259,4585315.0,1.0,17.3,17.3,0.0,2.0,2.0,6.0,1.0,1.0,0.0,0.0,4.083880
4,ЦАО,151004941,4814581.0,1.0,17.3,10.0,0.0,2.0,5.0,6.0,1.0,1.0,1.0,0.0,4.083880
5,ЦАО,150346368,4814581.0,1.0,17.3,17.3,0.0,2.0,6.0,5.0,1.0,0.0,1.0,0.0,4.131503
6,ЦАО,149031296,5225953.0,1.0,18.1,18.1,0.0,4.0,4.0,6.0,1.0,1.0,0.0,0.0,4.083880
7,ЦАО,150640194,5300000.0,1.0,32.0,20.0,7.0,1.0,9.0,4.0,1.0,0.0,0.0,0.0,4.254893
8,ЦАО,151004981,5377566.0,1.0,18.3,10.0,0.0,5.0,5.0,6.0,1.0,1.0,1.0,0.0,4.083880
9,ЦАО,148794719,5496232.0,1.0,23.7,23.7,0.0,2.0,6.0,5.0,1.0,0.0,1.0,0.0,4.131503


In [38]:
data.to_csv('C:/Users/Egor/Desktop/CMF/cian_pars.csv')